#### Verilerin yüklenmesi

In [1]:
from sklearn import set_config

set_config(display="diagram")

import pandas as pd

adult_census = pd.read_csv("dataset/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
target

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
48837     <=50K
48838      >50K
48839     <=50K
48840     <=50K
48841      >50K
Name: class, Length: 48842, dtype: object

#### Veriye ön bakış

In [2]:
data = adult_census.drop(columns=[target_name, "education-num"])
data.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States


#### train_test olarak datayı ayırmak

In [3]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

#### ColumnTransformerı ayarlama

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

#### Pipeline oluşturma

In [5]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4)),
])

model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat_preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])

#### loguniform ile rastgele sayı üretme

scipy.stats.loguniform kayan sayılar oluşturmak için kullanılabilir. Tamsayı değerli parametreler (örneğin min_samples_leaf) için rasgele değerler oluşturmak için aşağıdaki gibi uyarlayabiliriz:

In [6]:
from scipy.stats import loguniform


class loguniform_int:
    """Integer valued version of the log-uniform distribution"""
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

#### RandomizedSearchCV ile hyper parametre arama

In [7]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'classifier__l2_regularization': loguniform(1e-6, 1e3),
    'classifier__learning_rate': loguniform(0.001, 10),
    'classifier__max_leaf_nodes': loguniform_int(2, 256),
    'classifier__min_samples_leaf': loguniform_int(1, 100),
    'classifier__max_bins': loguniform_int(2, 255),
}

model_random_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=10,
    cv=5, verbose=1,
)
model_random_search.fit(data_train, target_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                sparse_threshold=0,
                                                                transformers=[('cat_preprocessor',
                                                                               OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                              unknown_value=-1),
                                                                               ['workclass',
                                                                                'education',
                                                                                'marital-status',
                                                                                'occupation',
                                                                                'relationship',
                                                                                'race',
                                                                                'sex',
                                                                                'native-country'])])),
                                             ('classifier',
                                              Hi...
                   param_distributions={'classifier__l2_regularization': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fbcfed2ef50>,
                                        'classifier__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fbcfddd2b90>,
                                        'classifier__max_bins': <__main__.loguniform_int object at 0x7fbcfddd3fa0>,
                                        'classifier__max_leaf_nodes': <__main__.loguniform_int object at 0x7fbcfddd3a30>,
                                        'classifier__min_samples_leaf': <__main__.loguniform_int object at 0x7fbcfddd2710>},
                   verbose=1)

#### Model skoru

In [8]:
accuracy = model_random_search.score(data_test, target_test)

print(f"The test accuracy score of the best model is "
      f"{accuracy:.2f}")

The test accuracy score of the best model is 0.88


#### En iyi hyper parametreler

In [9]:
from pprint import pprint

print("The best parameters are:")
pprint(model_random_search.best_params_)

The best parameters are:
{'classifier__l2_regularization': 0.06426955054851975,
 'classifier__learning_rate': 0.37749142223409426,
 'classifier__max_bins': 239,
 'classifier__max_leaf_nodes': 64,
 'classifier__min_samples_leaf': 45}


#### Sonuçların görselleştirilmesi

In [10]:
# get the parameter names
column_results = [
    f"param_{name}" for name in param_distributions.keys()]
column_results += [
    "mean_test_score", "std_test_score", "rank_test_score"]

cv_results = pd.DataFrame(model_random_search.cv_results_)
cv_results = cv_results[column_results].sort_values(
    "mean_test_score", ascending=False)

def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name

cv_results = cv_results.rename(shorten_param, axis=1)
cv_results

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
5,0.06427,0.377491,64,45,239,0.866971,0.003626,1
7,0.116381,0.880744,7,32,7,0.838416,0.003315,2
8,360.429456,0.036663,52,2,5,0.822527,0.002039,3
2,0.001173,0.506397,83,1,4,0.808031,0.001417,4
4,0.008448,0.043095,11,13,3,0.806557,0.001891,5
6,0.00032,0.085846,7,4,2,0.802080,0.002952,6
9,0.000687,0.009878,3,6,220,0.796648,0.001865,7
0,0.000001,0.001694,87,24,79,0.758947,0.000013,8
3,176.364745,0.001715,6,1,2,0.758947,0.000013,8
1,1.165844,7.09024,4,29,73,0.283476,0.005123,10


#### Hyper parametre tuning (randomized search) ve model ile fit edilen sonuçların kaydedilmesi

In [13]:
model_random_search = RandomizedSearchCV(
     model, param_distributions=param_distributions, n_iter=500,
     n_jobs=2, cv=5)
model_random_search.fit(data_train, target_train)
cv_results =  pd.DataFrame(model_random_search.cv_results_)
cv_results.to_csv("dataset/randomized_search_results.csv")

KeyboardInterrupt: 

In [14]:
cv_results = pd.read_csv("dataset/randomized_search_results.csv",
                         index_col=0)

(cv_results[column_results].rename(
    shorten_param, axis=1).sort_values("mean_test_score", ascending=False))

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
208,0.011775,0.076653,24,2,155,0.871393,0.001588,1
343,0.000404,0.244503,15,15,229,0.871339,0.002741,2
21,4.994918,0.077047,53,7,192,0.870793,0.001993,3
328,2.036232,0.224702,28,49,236,0.869837,0.000808,4
327,4.733808,0.036786,61,5,241,0.869673,0.002417,5
...,...,...,...,...,...,...,...,...
232,0.000097,9.976823,28,5,3,0.448205,0.253714,496
413,0.000001,8.828574,64,1,144,0.448205,0.253714,497
344,0.000003,7.091079,5,1,95,0.448205,0.253714,497
200,0.000444,6.236325,2,2,30,0.344629,0.207156,499
